In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .enableHiveSupport() \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# load json 

In [2]:
df = spark.read.json("file:///opt/spark/examples/src/main/resources/people.json")
df.show()

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [3]:
df.printSchema()
df.select("name").show()
df.select(df['name'], df['age'] + 1).show()
df.filter(df['age'] > 21).show()
df.groupBy("age").count().show()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+

+-------+---------+
|   name|(age + 1)|
+-------+---------+
|Michael|     NULL|
|   Andy|       31|
| Justin|       20|
+-------+---------+

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|NULL|    1|
|  30|    1|
+----+-----+



In [4]:
df.createOrReplaceTempView("people")

sqlDF = spark.sql("SELECT * FROM people")
sqlDF.show()

# spark.newSession().sql("SELECT * FROM global_temp.people").show()  # 会报错

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [5]:
df.createOrReplaceGlobalTempView("people")

sqlDF = spark.sql("SELECT * FROM people").show()

spark.newSession().sql("SELECT * FROM global_temp.people").show()

23/11/28 02:51:03 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/11/28 02:51:03 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/11/28 02:51:04 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



# load text

In [6]:
from pyspark.sql.functions import split

In [7]:
adj = spark.read.text('file:///home/jupyter/data/adj_rs1.txt')

In [8]:

columns = ['time', 'app_id', 'store', 'adid', 'openid', 'activity_kind', 'created_at', 'installed_at', 'reattributed_at', 'network_name', 'country', 'device_name', 'device_type', 'os_name', 'timezone', 'event_name', 'revenue_float', 'revenue', 'currency', 'revenue_usd', 'reporting_revenue']
adj_split = adj.select(split(adj['value'], ',').alias('split'))
adj_final = adj_split.select([adj_split['split'][i].alias(columns[i]) for i in range(len(columns))])

In [11]:
adj_final.write.saveAsTable('adj', mode='overwrite')

23/11/28 02:52:00 WARN TaskSetManager: Lost task 0.0 in stage 13.0 (TID 20) (worker2 executor 1): org.apache.spark.SparkException: [TASK_WRITE_FAILED] Task failed while writing rows to hdfs://master/user/hive/warehouse/adj.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.taskFailedWhileWritingRowsError(QueryExecutionErrors.scala:774)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.executeTask(FileFormatWriter.scala:420)
	at org.apache.spark.sql.execution.datasources.WriteFilesExec.$anonfun$doExecuteWrite$1(WriteFiles.scala:100)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:890)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2$adapted(RDD.scala:890)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:93)
	at org.apac

Py4JJavaError: An error occurred while calling o159.saveAsTable.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 13.0 failed 4 times, most recent failure: Lost task 0.3 in stage 13.0 (TID 24) (worker2 executor 1): org.apache.spark.SparkException: [TASK_WRITE_FAILED] Task failed while writing rows to hdfs://master/user/hive/warehouse/adj.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.taskFailedWhileWritingRowsError(QueryExecutionErrors.scala:774)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.executeTask(FileFormatWriter.scala:420)
	at org.apache.spark.sql.execution.datasources.WriteFilesExec.$anonfun$doExecuteWrite$1(WriteFiles.scala:100)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:890)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2$adapted(RDD.scala:890)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoin